This notebook is made to explore different ways of text augmentation. <hr>

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
from textaugment import EDA

In [3]:
t = EDA()

# Synonym replacement
def augment_by_sr(df, output_file_suff, n=2):
    _df = df.copy()
    _df['description'] = _df['description'].apply(lambda text: t.synonym_replacement(text, n=n))
    _df["word_count"] = _df["description"].apply(lambda s: len(s.split()))
    _df["taskId"] = _df["taskId"] + "_SR"
    _df.to_csv("data/augmented_SR_"+output_file_suff+".csv", index_label=False)

# Random insertion
def augment_by_ri(df, output_file_suff, n=2):
    _df = df.copy()
    _df['description'] = _df['description'].apply(lambda text: t.random_insertion(text, n=n))
    _df["word_count"] = _df["description"].apply(lambda s: len(s.split()))
    _df["taskId"] = _df["taskId"] + "_RI"
    _df.to_csv("data/augmented_RI_"+output_file_suff+".csv", index_label=False)

# Random swap
def augment_by_rs(df, output_file_suff):
    _df = df.copy()
    _df['description'] = _df['description'].apply(lambda text: t.random_swap(text))
    _df["word_count"] = _df["description"].apply(lambda s: len(s.split()))
    _df["taskId"] = _df["taskId"] + "_RS"
    _df.to_csv("data/augmented_RS_"+output_file_suff+".csv", index_label=False)

# Random deletion
def augment_by_rd(df, output_file_suff, p=0.4):
    _df = df.copy()
    _df['description'] = _df['description'].apply(lambda text: t.random_deletion(text, p=p))
    _df["word_count"] = _df["description"].apply(lambda s: len(s.split())) 
    _df["taskId"] = _df["taskId"] + "_RD"
    _df.to_csv("data/augmented_RD_"+output_file_suff+".csv", index_label=False)

In [4]:
# Back Translation
def augment_by_bt(df, output_file_suff):
    if output_file_suff == "EN": 
        from_model_name = 'facebook/wmt19-en-de'
        to_model_name = 'facebook/wmt19-de-en'
    elif output_file_suff == "DE": 
        from_model_name = 'facebook/wmt19-de-en' 
        to_model_name = 'facebook/wmt19-en-de'
    else:
        raise "Bad output_file_suff"
        
    back_translation_aug = naw.BackTranslationAug(
        from_model_name=from_model_name,
        to_model_name=to_model_name
        )
    _df = df.copy()
    _df['description'] = _df['description'].apply(lambda text: back_translation_aug.augment(text)[0])
    _df["word_count"] = _df["description"].apply(lambda s: len(s.split())) 
    _df["taskId"] = _df["taskId"] + "_BT"
    _df.to_csv("data/augmented_BT_"+output_file_suff+".csv", index_label=False)

<hr>

In [5]:
df_en = pd.read_csv("data/open_tasks_EN.csv").dropna(subset=["description"]) 
df_de = pd.read_csv("data/open_tasks_DE.csv").dropna(subset=["description"]) 

In [6]:
len(df_de), len(df_en)

(1556, 299)

In [7]:
# remove test descriptions before augmentation
df_en = df_en[df_en["word_count"] > 4]
df_de = df_de[df_de["word_count"] > 4]

In [8]:
for df, suff in zip([df_en, df_de], ["EN", "DE"]):
    augment_by_sr(df, suff)
    augment_by_ri(df, suff) 
    augment_by_rs(df, suff)
    augment_by_rd(df, suff)
    augment_by_bt(df, suff)

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-de-en and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-de-en and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

<hr>

https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb